# ToBig's_11th_W05_Neural_Network_실습
## Assignment_01 : 주석달기
### 11기 김대웅

In [1]:
import numpy as np

In [2]:
# 행렬 X에 가중치 벡터 W를 내적해 Hidden layer H 생성
# <=> H = w_1 * x_1 + w_2 * x_2 + ... + w_d * x_d
def linear(X, W):
    H = np.dot(X, W)
    return H

In [3]:
# activation function : 시그모이드 함수
def sigmoid(H):
    p = 1 / (1 + np.exp(-H))
    return p

In [4]:
# 입력 Feature는 1 X 2 행렬
X = np.array([1, 2])

In [5]:
#가중치 행렬 W는 1 X 2 행렬
W = np.array([2, 3])

In [6]:
# 입력 Feature x_i에 각각 가중치 w_i를 곱하여 Hidden layer H 계산 (1 * 1)
# H값이 양수일 경우. Sigmoid는 1에 가까운 값
H = linear(X, W)
print(H)
# H에 activation function : sigmoid
p = sigmoid(H)
print(p)

8
0.9996646498695336


In [7]:
W = np.array([-4, -3])

In [8]:
# H값이 음수일 경우, Sigmoid는 0에 가까운 값
H = linear(X, W)
print(H)
p = sigmoid(H)
print(p)

-10
4.5397868702434395e-05


In [9]:
# Iris 자료 load
from sklearn.datasets import load_iris

In [10]:
data = load_iris()

In [11]:
x = data["data"]
y = data["target"]

In [14]:
len(x)

150

In [12]:
# X (150, 4)
x[:10]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1]])

In [13]:
# y (150, )
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [20]:
class Softmax:
    def __init__(self):
        # Parameter 저장할 Dict
        self.params = {}
        # 임의 가중치 행렬 W (4 x 3) (Hidden layer의 Node 수 : 3개)
        self.params['W'] = 0.0001 * np.random.randn(4, 3)
        # Bias 항으로 초기값 1을 부여함. 
        # WX가 (150 X 3) 행렬이므로 각 bias의 개수는 hidden layer의 Node 수(3)와 동일 
        self.params['b'] = np.ones(3)
    
    def forward(self, X):
        # parameter dict에서 Weight와 Bias를 가져와 local variable W와 b에 할당
        W = self.params['W']
        b = self.params['b']

        # Hidden layer = Weight * Features  + Bias // H = XW + b
        # H : (150, 3)
        h = np.dot(X, W) + b
        # Softmax probability 계산을 위해 지수함수 변환 : exp(H)
        # A : (150, 3)
        a = np.exp(h)
        # stable_a = np.exp(h - np.max(h, axis = 1).reshape(-1,1))
        # 각각의 probability.
        # P : (150, 3)
        p = a/np.sum(a, axis = 1, keepdims=True)
        # cf. keepdims = True : a의 원래 형태 유지. (열벡터)
        return p
    
    # Softmax-with-loss Node
    def loss(self, X, T):
        # 예측확률 p를 계산
        p = self.forward(X)
        # 전체 자료 길이 n 
        n = T.shape[0]
        
        # Cross Entropy 오차의 계산 
    
        # Error = -log(정답 label의 Softmax probability)의 총 합계
        # log내에 들어가는 확률값이 1에 가까울수록 -log변환의 값은 작다.
        # 즉 정답레이블에 대해 정답으로 예측할 확률이 클수록(1에 가까울수록)
        # -log(Softmax probability)는 0에 가깝기 때문에 이렇게 설정
        
        log_likelihood = 0
        log_likelihood -= np.log(p[np.arange(n), T]).sum()
        # loss를 Sample Size N으로 나눔 (개별 자료에 대한 loss를 나타내기 위해)
        Loss = log_likelihood / n

        return Loss
    
    def accuracy(self, X, T):
        p = self.forward(X) # Softmax prob : P (150, 3)
        # predict (150, 1) : 분류 결과 : 가장 큰 확률값을 가지는 index
        predict = np.argmax(p, axis = 1) #예측 결과 index 1darray 로 출력 
        
        # 정분류율(Accuracy) = 1 - (오분류율)
        return 1 - np.count_nonzero(predict - T)/len(T)
        
    def gradient(self, X, T, learning_rate = 0.0001):
        # Softmax prob 계산 p에 할당
        p = self.forward(X)

        # Softmax-Loss Node의 입력값(H) 각각의 Loss에 대한 Gradient
        # (H : 150, 3)
        # (P : 150, 3)
        # dp (150, 3) : derivate of Loss w.r.t. h_i ( i = 1, 2, ... , 150 )
        dp = p.copy()
        dp[np.arange(len(T)), T] -= 1
        
        # Softmax-loss의 입력단 : h_1, h_2, h_3 각각의 Gradient를 구합니다.
        # 이 때 각 자료에 대해서 Gradient값이 구해지며, update를 위해선 150개 자료의 각 Gradient의 평균값을 사용합니다.
        # 각 Gradient의 평균값을 구하기 위해 batch size로 나눠줍니다(뒤에서 총합을 내어 mean이 됨)
        # cf. 여기서는 전체 자료를 사용하기 때문에 batch size = 150. SGD의 mini bath의 경우 mini batch size로 나누어줌
        
        dp /= len(T)
        
        #목적함수에 대한 가중치 미분값을 담을 zero array 생성
        grads = {}
        
        #목적함수에 대한 가중치 미분값 합 구하기
        # {partial L} over {partial W} = [X^T]product dot[{partial L} over {partial H}]
        # X : (150, 4), dp : (150, 3)
        # grads["W"] : (4, 3)
        #     row : input feature 4
        #     col : Softmax-loss node input 3
        
        grads['W'] = np.dot(X.T, dp)
        # {partial L} over {partial b} = 1 * [{partial L} over {partial H} = dp]
        # grads["b"] : (1, 3)
        grads['b'] = np.sum(dp, axis = 0)
        #p-t 대신 dp 사용 가능
        
        # parameter update 
        self.params['W'] -= learning_rate * grads['W']
        self.params['b'] -= learning_rate * grads['b']

In [21]:
softmax = Softmax()

In [22]:
for i in range(1000000000):
    softmax.gradient(x, y)
    
    if i % 1000 == 0:
        print(i, "번째 학습중입니다.")
        print("Accuracy : ", softmax.accuracy(x, y))
        print("Loss :     ", softmax.loss(x, y))
        print(softmax.params['W'])

0 번째 학습중입니다.
Accuracy :  0.33333333333333337
Loss :      1.0987191201522015
[[ 4.71958896e-05  4.22167250e-05 -2.19067037e-04]
 [-2.01014883e-04 -4.52424608e-05 -9.59231972e-05]
 [ 1.26014621e-04  1.63714011e-04  1.26605259e-04]
 [ 1.20505533e-04 -1.07729572e-05  3.49531937e-05]]
1000 번째 학습중입니다.
Accuracy :  0.33999999999999997
Loss :      1.0202038815448475
[[-0.00317786  0.00063262  0.00241559]
 [ 0.02382499 -0.0107081  -0.01345908]
 [-0.05771192  0.01480966  0.04331859]
 [-0.02531247  0.00348564  0.02197151]]
2000 번째 학습중입니다.
Accuracy :  0.6666666666666667
Loss :      0.9647084317168441
[[ 0.00863864 -0.00038477 -0.00838352]
 [ 0.05390199 -0.02184066 -0.03240351]
 [-0.10180242  0.02763717  0.07458158]
 [-0.04582038  0.00630861  0.03965645]]
3000 번째 학습중입니다.
Accuracy :  0.6666666666666667
Loss :      0.9164501352728479
[[ 0.02158507 -0.00161939 -0.02009533]
 [ 0.08303009 -0.03268098 -0.0506913 ]
 [-0.14174623  0.03947414  0.10268843]
 [-0.06455972  0.00871589  0.05598851]]
4000 번째 학습중입니

KeyboardInterrupt: 